In [5]:
from google.colab import files
!rm -rf ./datasets

In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt
%pip install -q roboflow

import torch
import os
import shutil
from IPython.display import Image, clear_output


fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [6]:
!pip install roboflow

os.environ["DATASET_DIRECTORY"] = "/content/YoloFormatDatasets"
yolov5_directory = '/content/yolov5'
os.environ["YOLOV5_DIRECTORY"] = yolov5_directory

from roboflow import Roboflow
rf = Roboflow(api_key='TiExgP2alzxgNpQViv9O')
project = rf.workspace().project('maskdectector2')
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /content/YoloFormatDatasets/MaskDectector2-1 in yolov5pytorch:: 100%|██████████| 1795/1795 [00:00<00:00, 6101.35it/s]


In [7]:
yolov5_model = 's'
config = 'yolov5' + yolov5_model + '.yaml'
startWeights = 'yolov5' + yolov5_model + '.pt'

os.environ["YOLO_CONFIG"] = config
os.environ["INIT_WEIGHTS"] = startWeights

In [8]:
!python train.py --img 416 --batch-size -1 --epochs 300 --data {dataset.location}/data.yaml --cfg {config} --cache

2024-04-19 00:43:53.448253: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-19 00:43:53.448305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-19 00:43:53.450291: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=/content/YoloFormatDatasets/MaskDectector2-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=-1, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=F

In [9]:
def getLastCreatedExp():
  """
    Returns name of last created exp folder (located in /runs/train/...)
  """
  contents = os.listdir(f'{yolov5_directory}/runs/train')
  contentsWithoutZIP = []
  for f in contents:
    if '.zip' not in f and 'exp' in f:
      contentsWithoutZIP.append(f)
  contentsWithoutZIP.sort()
  return contentsWithoutZIP[-1]

last_exp = getLastCreatedExp()



In [10]:
# evaluates trained model on test set
!python val.py --data {dataset.location}/data.yaml --batch-size 8 --weights {yolov5_directory}/runs/train/{last_exp}/weights/last.pt

val: data=/content/YoloFormatDatasets/MaskDectector2-1/data.yaml, weights=['/content/yolov5/runs/train/exp3/weights/last.pt'], batch_size=8, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-304-g22361691 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/YoloFormatDatasets/MaskDectector2-1/valid/labels.cache... 100 images, 0 backgrounds, 0 corrupt: 100% 100/100 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class    

Weights and train results are located in `runs/train/exp*` whiel test results are saved in `runs/val/exp*`.

